In [86]:
import re
from time import time
import math
import torch
import fairseq
from fairseq import checkpoint_utils, data, options, tasks
from fairseq.sequence_generator import SequenceGenerator

import sys
sys.path.append('..')
from pythainlp.tokenize import word_tokenize


load dataset


In [87]:
# from Wang -- 2019-06-26_output_Non_Empty_Answer.txt
pattern = r'answer: \[\'(.+)\'\] variable: (\w.+)'
texts = {
    'th': [],
    'en': []
}

with open('../data/wang/2019-06-26_output_Non_Empty_Answer.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
#         print(line)
#         print(line)
        try:
            match_obj = re.search(pattern, line)

            th_text = match_obj[1]
            if th_text[0] == '?':
                th_text = th_text[1:]
            en_text = match_obj[2]

            texts['th'].append(th_text)
            texts['en'].append(en_text)
        except Exception as e:
#             print(e)
            continue
    
    print('number of examples = ', len(texts['th']))


number of examples =  6118


In [88]:
# texts_test_set = {
#     'th': ['ฉันรักคุณ'],
#     'en': ['I love you']
# }

In [89]:
# Define BPE
from bpemb import BPEmb

bpemb_pretrained ={
    'th': {
        '25000': BPEmb(lang="th", vs=25000)
    },
    'en': {
        '25000': BPEmb(lang="en", vs=25000)
    }
}

def encode_bpe(sentences, lang, n_vocab=25000):
    """Return a list of bpe tokens give a list of sentences"""
    segmented_sentences = []
    for sentence in sentences:
#         print(sentence)
        bpe_tokens = bpemb_pretrained[lang]['{}'.format(n_vocab)].encode(sentence)
        segmented_sentences.append(' '.join(bpe_tokens))
        
    return segmented_sentences


In [136]:
def run_inference(src_lang, tgt_lang,
                  src_tok_type, tgt_tok_type,
                  src_dict_path, tgt_dict_path,
                  model_path,
                  examples=texts,
                  n_examples=10,
                  beam_size=5):
    
    
#     model_path = model_path or '../data/opensubtitles_model/exp003-1/transformer_base/checkpoint_best.pt'
    print('| loading model from {}'.format(model_path))

    state = checkpoint_utils.load_checkpoint_to_cpu(model_path)
    args = vars(state['args'])
    args['data'] = '/root/mt-opus/' + args['data']
    print('args', args)
    
    ensemble, _, _ = checkpoint_utils.load_model_ensemble_and_task([model_path], arg_overrides=args, task=None)
    
    src_dict = data.Dictionary()
    src_dict.add_from_file(src_dict_path)

    tgt_dict = data.Dictionary()
    tgt_dict.add_from_file(tgt_dict_path)
    
    print('len(src_dict) = ', len(src_dict))
    print('len(tgt_dict) = ', len(tgt_dict))


    _generator = SequenceGenerator(tgt_dict=tgt_dict, beam_size=beam_size,
                                   len_penalty=-100.0,
                                  max_len_b=50)
    
    print('Inference from {} to {}'.format(src_lang, tgt_lang))
    for i, src in enumerate(examples[src_lang][:n_examples]):
        
        print(i+1)
        print('Source :', src)
        if src_tok_type == 'newmm':
            toks = word_tokenize(src, engine='newmm', keep_whitespace=False)
        elif src_tok_type == 'sentencepiece':
            toks = encode_bpe([src], lang=src_lang)[0]
        else:
            raise "error"
        print('tokens :', toks)
        src_indices = [src_dict.index(t) for t in toks]
        src_len = [len(src_indices)]

        src_indices_tensor = torch.LongTensor(src_indices).view(1, -1)
        src_len_tensor = torch.LongTensor(src_len).view(1, -1)

#         print('src indices:', src_indices_tensor.size(), src_indices_tensor)
    #     print('src len    :', src_len_tensor.size(), src_len_tensor)

        sample = {
            'net_input': {
                    'src_tokens': src_indices_tensor, 'src_lengths': src_len_tensor,
            },
        }
        
#         print('sample', sample)
        hypos = _generator.generate(models=ensemble, sample=sample)
    
        print('\nTarget      :', examples[tgt_lang][i])
        if tgt_tok_type == 'newmm':
            toks = ' '.join(word_tokenize(examples[tgt_lang][i], keep_whitespace=""))
        elif tgt_tok_type == 'sentencepiece':
            toks = encode_bpe([examples[tgt_lang][i]], lang=tgt_lang)[0]
        else:
            raise "error"
        print('Target toks : {}'.format(toks))
        print('')
        
        for i in range(0, min(5, beam_size)):
            hypo = hypos[0][i]
            indices = list(hypo['tokens'].clone().detach().numpy())

            out_symbols = ' '.join([tgt_dict[idx] for idx in indices])
            print('Prediction {}: {}'.format(i+1, out_symbols))

        print('\n\n')

    
# # print(task)

In [135]:
# exp002-1.1 en-th word-word

model_path = '../data/opensubtitles_model/exp002-1/transformer_base/checkpoint15.pt'
src_dict_path = '../data/opensubtitles_bin/newmm-newmm/en-th/dict.en.txt'
tgt_dict_path = '../data/opensubtitles_bin/newmm-newmm/en-th/dict.th.txt'

test_src_tok_path = '../data/opensubtitles_tok/newmm-newmm/en-th/test.en'
test_tgt_tok_path = '../data/opensubtitles_tok/newmm-newmm/en-th/test.th'

texts_test_sample = {
    'th': ['a',' a','a'],
    'en': ["Michael dropped his fork and aimed daggers at me. \"Are you kidding me, Jill? He's a miserable, sarcastic punk.\"",
           "Now he is here,\" I exclaimed. \"For Heaven's sake, hurry down! Do be quick; and stay among the trees till he is fairly in",
           "I just mean that the human auditory system isn’t an absolute acoustic instrument.",
           
          ]}

with open(test_src_tok_path, 'r') as f:
    for item in f.readlines():
        texts_test_set['en'].append(item.replace('\n', ''))
with open(test_tgt_tok_path, 'r') as f:
    for item in f.readlines():
        texts_test_set['th'].append(item.replace('\n', ''))

    
print('model_path', model_path)
print('src_dict_path', src_dict_path)
print('tgt_dict_path', tgt_dict_path)


# run_inference(src_lang='en', tgt_lang='th',
#               src_tok_type='newmm', tgt_tok_type='newmm',
#               src_dict_path=src_dict_path, tgt_dict_path=tgt_dict_path,
#               model_path=model_path,
#               examples=texts_test_sample,
#               n_examples=1000,
#               beam_size=5)



# run_inference(src_lang='en', tgt_lang='th',
#               src_tok_type='newmm', tgt_tok_type='newmm',
#               src_dict_path=src_dict_path, tgt_dict_path=tgt_dict_path,
#               model_path=model_path,
#               examples=texts,
#               n_examples=300,
#               beam_size=10)


run_inference(src_lang='en', tgt_lang='th',
              src_tok_type='newmm', tgt_tok_type='newmm',
              src_dict_path=src_dict_path, tgt_dict_path=tgt_dict_path,
              model_path=model_path,
              examples=texts_test_set,
              n_examples=1000,
              beam_size=5)

model_path ../data/opensubtitles_model/exp002-1/transformer_base/checkpoint15.pt
src_dict_path ../data/opensubtitles_bin/newmm-newmm/en-th/dict.en.txt
tgt_dict_path ../data/opensubtitles_bin/newmm-newmm/en-th/dict.th.txt
| loading model from ../data/opensubtitles_model/exp002-1/transformer_base/checkpoint15.pt
args {'no_progress_bar': True, 'log_interval': 250, 'log_format': 'simple', 'tensorboard_logdir': '', 'tbmf_wrapper': False, 'seed': 1, 'cpu': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'criterion': 'label_smoothed_cross_entropy', 'tokenizer': 'space', 'bpe': None, 'optimizer': 'adam', 'lr_scheduler': 'inverse_sqrt', 'task': 'translation', 'num_workers': 1, 'skip_invalid_size_inputs_valid_test': False, 'max_tokens': 3000, 'max_sentences': None, 'required_batch_size_multiple': 8, 'dataset_impl': None, 'train_subset': '


Target      : บ ริ ค ลูกคิด ยังไง เรื่อง แผน เกม พรุ่งนี้
Target toks : บ ริ ค ลูกคิด ยังไง เรื่อง แผน เกม พรุ่งนี้

Prediction 1: บ ริ ค ลูก คิด ว่า อะไร คือ กุญแจ เกม พรุ่งนี้ </s>
Prediction 2: บ ริ ค ลูก คิด ว่า อะไร คือ กุญแจ สำคัญ ของ เกม พรุ่งนี้ </s>
Prediction 3: บ ริ ค ลูก คิด ว่า อะไร คือ กุญแจ สำคัญ ของ เกม วันพรุ่งนี้ </s>
Prediction 4: บ ริ ค ลูก คิด ว่า อะไร คือ กุญแจ สำคัญ ของ เกม พรุ่งนี้ ? </s>
Prediction 5: บ ริ ค ลูก คิด ว่า อะไร จะ เป็น กุญแจ สำคัญ ของ เกม พรุ่งนี้ </s>



11
Source : Because all of it sounds like me .
tokens : ['Because', 'all', 'of', 'it', 'sounds', 'like', 'me', '.']

Target      : เพราะ ทุก อย่างนั้น คือ ผม
Target toks : เพราะ ทุก อย่างนั้น คือ ผม

Prediction 1: เพราะ ทุกอย่าง มัน ฟัง ดูเหมือน เป็น เหมือน ฉัน </s>
Prediction 2: เพราะ ทุกอย่าง มัน ฟัง ฟัง ฟัง เหมือน ฉัน </s>
Prediction 3: เพราะ ทุกอย่าง มัน ฟัง ฟัง เหมือน เป็น เหมือน ฉัน </s>
Prediction 4: เพราะ ทุกอย่าง มัน ฟัง ฟัง ฟัง ฟัง เหมือน ฉัน </s>
Prediction 5: เพราะ ทุกอย่าง มัน ฟัง ฟ


Target      : นั่นไง เขา มา แล้ว !
Target toks : นั่นไง เขา มา แล้ว !

Prediction 1: ใช่ เขา อยู่ นั่น เขา อยู่ นั่น ใช่ เขา อยู่ นั่น เขา อยู่ นั่น ใช่ เขา อยู่ นั่น เขา อยู่ นั่น </s>
Prediction 2: ใช่ เขา อยู่ นั่น เขา อยู่ นั่น ใช่ เขา อยู่ นั่น เขา อยู่ นั่น ใช่ เขา อยู่ นั่น เขา อยู่ นั่น เขา อยู่ นั่น </s>
Prediction 3: ใช่ เขา อยู่ นั่น เขา อยู่ นั่น ใช่ เขา อยู่ นั่น เขา อยู่ นั่น ใช่ เขา อยู่ นั่น เขา อยู่ นั่น ใช่ เขา อยู่ นั่น </s>
Prediction 4: ใช่ เขา อยู่ นั่น เขา อยู่ นั่น ใช่ เขา อยู่ นั่น เขา อยู่ นั่น ใช่ เขา อยู่ นั่น เขา อยู่ นั่น ใช่ เขา เป็น เขา </s>
Prediction 5: ใช่ เขา อยู่ นั่น เขา อยู่ นั่น ใช่ เขา อยู่ นั่น เขา อยู่ นั่น ใช่ เขา อยู่ นั่น เขา อยู่ นั่น ใช่ เขา เป็น เขา เขา </s>



26
Source : What do you think ?
tokens : ['What', 'do', 'you', 'think', '?']

Target      : แล้ว คุณ คิด ว่า ไง ?
Target toks : แล้ว คุณ คิด ว่า ไง ?

Prediction 1: คิด ว่า ไง </s>
Prediction 2: นาย คิด ว่า ไง </s>
Prediction 3: คิด ว่า ไง ? </s>
Prediction 4: คิด ว่า ไง ล่ะ </s>


Target      : หมาย ถึงว่า สอน ชกมวย
Target toks : หมาย ถึงว่า สอน ชกมวย

Prediction 1: โช โช </s>
Prediction 2: ฉัน รู้ </s>
Prediction 3: โช โช โช </s>
Prediction 4: โช โช โช โช </s>
Prediction 5: โช โช โช โช โช </s>



45
Source : Thank you for sitting here for an hour .
tokens : ['Thank', 'you', 'for', 'sitting', 'here', 'for', 'an', 'hour', '.']

Target      : ขอบคุณ นะคะ ที่นั่ง อยู่ ที่นี่ เป็น ชั่วโมง
Target toks : ขอบคุณ นะคะ ที่นั่ง อยู่ ที่นี่ เป็น ชั่วโมง

Prediction 1: ขอบคุณ ที่ รอ ผม สัก ชั่วโมง </s>
Prediction 2: ขอบคุณ สำหรับ การ นั่ง ที่นี่ สำหรับ หนึ่ง ชั่วโมง </s>
Prediction 3: ขอบคุณ สำหรับ การ นั่ง ที่นี่ สำหรับ ชั่วโมง หนึ่ง </s>
Prediction 4: ขอบคุณ สำหรับ การ นั่ง ที่นี่ สำหรับ หนึ่ง ชั่วโมง . </s>
Prediction 5: ขอบคุณ สำหรับ การ นั่ง ที่นี่ สำหรับ หนึ่ง ชั่วโมง ค่ะ </s>



46
Source : - Cristina , it ' s so complicated . - I can ' t ... you know , it ' s okay . Just ... fine .
tokens : ['-', 'Cristina', ',', 'it', "'", 's', 'so', 'complicated', '.', '-', 'I', 


Target      : ทำไม เธอ ถึง ทำ แบบนี้
Target toks : ทำไม เธอ ถึง ทำ แบบนี้

Prediction 1: ทำไม คุณ ทำ แบบนี้ </s>
Prediction 2: ทำไม คุณ ถึง ทำ แบบนี้ </s>
Prediction 3: ทำไม คุณ ต้อง ทำ แบบนี้ ด้วย </s>
Prediction 4: ทำไม คุณ ถึง ทำ แบบนี้ ล่ะ </s>
Prediction 5: ทำไม คุณ ถึง มา ทำ แบบนี้ </s>



59
Source : That asshole !
tokens : ['That', 'asshole', '!']

Target      : ไอ้ สารเลว นั่น
Target toks : ไอ้ สารเลว นั่น

Prediction 1: แก ต้อง ตาย </s>
Prediction 2: แก ต้อง ไป </s>
Prediction 3: นี่ มัน อะไร กัน </s>
Prediction 4: นี่ มัน อะไร เนี่ย </s>
Prediction 5: นี่ มัน อะไร กัน เนี่ย </s>



60
Source : - It looks pretty cool .
tokens : ['-', 'It', 'looks', 'pretty', 'cool', '.']

Target      : - สุดยอด ไป เลย
Target toks : - สุดยอด ไป เลย

Prediction 1: มัน ดู สวย มาก </s>
Prediction 2: - มัน ดู สวย มาก </s>
Prediction 3: - มัน ดู น่ารัก มาก </s>
Prediction 4: - มัน ดู สวย ดี </s>
Prediction 5: - มัน ดู สวย มาก เลย </s>



61
Source : There he is . There he is .
tokens : ['There', '

...


Target      : ความจำเป็น ใน ระยะเวลา หนึ่ง น่ะ
Target toks : ความจำเป็น ใน ระยะเวลา หนึ่ง น่ะ

Prediction 1: ข้า หวัง ว่า จะ ได้ พบ ท่าน </s>
Prediction 2: ข้า หวัง ว่า จะ ได้ พบ กับ ท่าน </s>
Prediction 3: ข้า หวัง ว่า จะ ได้ พบ กับ เจ้า </s>
Prediction 4: ข้า หวัง ว่า จะ ได้ พบ กับ ท่าน แล้ว </s>
Prediction 5: ข้า หวัง ว่า จะ ได้ พบ กับ ท่าน นะ </s>



931
Source : Yeah , I guess .
tokens : ['Yeah', ',', 'I', 'guess', '.']

Target      : ก็ได้ มั้ง
Target toks : ก็ได้ มั้ง

Prediction 1: ผม ว่า </s>
Prediction 2: คง งั้น </s>
Prediction 3: ฉัน ว่า </s>
Prediction 4: ผม คิด ว่า </s>
Prediction 5: ฉัน คิด ว่า </s>



932
Source : When you ' re done helping your boyfriend , we need to talk .
tokens : ['When', 'you', "'", 're', 'done', 'helping', 'your', 'boyfriend', ',', 'we', 'need', 'to', 'talk', '.']

Target      : เมื่อ คุณ ช่วย แฟน ของ คุณ เสร็จ เมื่อไหร่ , เรา ต้อง คุย กัน .
Target toks : เมื่อ คุณ ช่วย แฟน ของ คุณ เสร็จ เมื่อไหร่ , เรา ต้อง คุย กัน .

Prediction 1: พอ เลิก ช่วย 


Target      : ประเด็น ล่ะ มัน ก็ แค่ ตำแหน่ง โง่ ๆ
Target toks : ประเด็น ล่ะ มัน ก็ แค่ ตำแหน่ง โง่ ๆ

Prediction 1: ประเด็น คือ อะไร มัน ก็ แค่ ตำแหน่ง โง่ ๆ </s>
Prediction 2: ประเด็น คือ อะไร มัน ก็ แค่ ตำแหน่ง โง่ ๆ เอง </s>
Prediction 3: ประเด็น คือ มัน เป็น แค่ ตำแหน่ง โง่ ๆ เป็น แค่ ตำแหน่ง </s>
Prediction 4: ประเด็น คือ อะไร มัน ก็ แค่ ตำแหน่ง โง่ ๆ เล่น ๆ </s>
Prediction 5: ประเด็น คือ อะไร มัน ก็ แค่ ตำแหน่ง โง่ ๆ ชื่อ ตำแหน่ง โง่ ๆ </s>



948
Source : You guys can share .
tokens : ['You', 'guys', 'can', 'share', '.']

Target      : แบ ง กัน ก ได
Target toks : แบ ง กัน ก ได

Prediction 1: เรื่อง ดี ๆ เรื่อง ที่ เรา จะ แบ่ง กัน แบ่ง เรา แบ่ง เรา แบ่ง เรา แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง ร่วม </s>
Prediction 2: เรื่อง ดี ๆ เรื่อง ที่ เรา จะ แบ่ง กัน แบ่ง เรา แบ่ง เรา แบ่ง เรา แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แบ่ง แ


Target      : อ้อ เมื่อวันก่อน ฉัน ว่า จะ แอด เธอ เป็นเพื่อน แต่ ฉัน หา โปรไฟล์ ไม่ เจอ เธอ ตั้งค่า ไว้ หรือ ยังไง
Target toks : อ้อ เมื่อวันก่อน ฉัน ว่า จะ แอด เธอ เป็นเพื่อน แต่ ฉัน หา โปรไฟล์ ไม่ เจอ เธอ ตั้งค่า ไว้ หรือ ยังไง

Prediction 1: ผม พยายาม จะ บอก คุณ เมื่อวันก่อน แต่ ผม หา ประวัติ คุณ ไม่ เจอ </s>
Prediction 2: ผม พยายาม จะ เพิ่ม คุณ วัน อื่นๆ แต่ ผม หา ประวัติ คุณ ไม่ เจอ </s>
Prediction 3: ผม พยายาม จะ บอก คุณ เมื่อวันก่อน แต่ ผม หา ประวัติ คุณ ไม่ เจอ เลย </s>
Prediction 4: ฉัน พยายาม จะ เพิ่ม คุณ วัน อื่นๆ แต่ ฉัน หา ประวัติ คุณ ไม่ เจอ </s>
Prediction 5: ผม พยายาม จะ เพิ่ม คุณ ลง ไป เมื่อวันก่อน แต่ ผม หา ประวัติ คุณ ไม่ เจอ </s>



964
Source : Hello ?
tokens : ['Hello', '?']

Target      : สวัสดี ?
Target toks : สวัสดี ?

Prediction 1: ว่า ไง </s>
Prediction 2: ว่า ไง ? </s>
Prediction 3: ว่า ไง นะ </s>
Prediction 4: มี อะไร เหรอ </s>
Prediction 5: ว่า ไง นะ ? </s>



965
Source : Unbelievable .
tokens : ['Unbelievable', '.']

Target      : ชั้น ล่ะ ไม่ อยาก จะ เ


Target      : บางที ถ้า เรา ทำ ตัวดี เขา อาจจะ โอ้ พระเจ้า
Target toks : บางที ถ้า เรา ทำ ตัวดี เขา อาจจะ โอ้ พระเจ้า

Prediction 1: บางที ถ้า เรา เป็น คนดี เขา จะ -- พระเจ้า </s>
Prediction 2: บางที ถ้า เรา เป็น คนดี เขา อาจจะ -- พระเจ้า </s>
Prediction 3: บางที ถ้า เรา เป็น คนดี เขา จะ -- โอ้ พระเจ้า </s>
Prediction 4: บางที ถ้า เรา เป็น คนดี เขา อาจจะ -- โอ้ พระเจ้า </s>
Prediction 5: บางที ถ้า เรา ทำ ตัวดี เขา จะ -- โอ้ พระเจ้า </s>



982
Source : Russ , the boys and I can get this done minus the bloodshed .
tokens : ['Russ', ',', 'the', 'boys', 'and', 'I', 'can', 'get', 'this', 'done', 'minus', 'the', 'bloodshed', '.']

Target      : รัส ส์.. ฉัน กับ เด็ก ๆ ทำงาน นี้ ได้ โดย ไม่ เสียเลือดเนื้อ เลย
Target toks : รัส ส์.. ฉัน กับ เด็ก ๆ ทำงาน นี้ ได้ โดย ไม่ เสียเลือดเนื้อ เลย

Prediction 1: รัส , เด็ก ๆ และ ผม ทำให้ เรื่อง นี้ จบ ได้ </s>
Prediction 2: รัส , เด็ก ๆ และ ฉัน สามารถ จัดการ เรื่อง นี้ ได้ </s>
Prediction 3: รัส , พวก เด็ก ๆ และ ผม ทำให้ เรื่อง นี้ จบ ได้ </s>
Predict


Target      : คราวนี้ ใคร พูด ไร้สาระ กัน แน่
Target toks : คราวนี้ ใคร พูด ไร้สาระ กัน แน่

Prediction 1: คิด ว่า ไง จะ บ้า เหรอ ? </s>
Prediction 2: คิด ว่า ไง จะ บ้า เหรอ บ้า เหรอ บ้า เหรอ จะ บ้า เหรอ ? </s>
Prediction 3: คิด ว่า ไง จะ บ้า เหรอ บ้า เหรอ บ้า เหรอ จะ บ้า เหรอ ไง </s>
Prediction 4: คิด ว่า ไง จะ บ้า เหรอ บ้า เหรอ บ้า เหรอ จะ บ้า เหรอ บ้า ได้ งั้น เหรอ </s>
Prediction 5: คิด ว่า ไง จะ บ้า เหรอ บ้า เหรอ บ้า เหรอ จะ บ้า เหรอ บ้า ได้ งั้น เหรอ ? </s>



999
Source : We ' re very excited to be here kicking off the JVC festival this year
tokens : ['We', "'", 're', 'very', 'excited', 'to', 'be', 'here', 'kicking', 'off', 'the', 'JVC', 'festival', 'this', 'year']

Target      : พวกเรา รู้สึก ตื่นเต้น มาก ใน การแสดง ที่ เทศกาล JVC ปี นี้
Target toks : พวกเรา รู้สึก ตื่นเต้น มาก ใน การแสดง ที่ เทศกาล JVC ปี นี้

Prediction 1: เรา ตื่นเต้น กัน มาก ที่จะ ได้ ออก ไป เล่น ด้วยกัน </s>
Prediction 2: เรา ตื่นเต้น กัน มาก ที่จะ ได้ ออก ไป ชม การแสดง ใน ปี นี้ </s>
Prediction 3: เรา ตื

In [137]:
# exp003-1.1 th-en word-word

model_path = '../data/opensubtitles_model/exp003-1/transformer_base/checkpoint10.pt'
src_dict_path = '../data/opensubtitles_bin/newmm-newmm/th-en/dict.th.txt'
tgt_dict_path = '../data/opensubtitles_bin/newmm-newmm/th-en/dict.en.txt'

test_src_tok_path = '../data/opensubtitles_tok/newmm-newmm/th-en/test.th'
test_tgt_tok_path = '../data/opensubtitles_tok/newmm-newmm/th-en/test.en'

texts_test_set = {
    'th': [],
    'en': []
}

with open(test_src_tok_path, 'r') as f:
    for item in f.readlines():
        texts_test_set['th'].append(item.replace('\n', ''))
with open(test_tgt_tok_path, 'r') as f:
    for item in f.readlines():
        texts_test_set['en'].append(item.replace('\n', ''))

    
print('model_path', model_path)
print('src_dict_path', src_dict_path)
print('tgt_dict_path', tgt_dict_path)

# run_inference(src_lang='th', tgt_lang='en',
#               src_tok_type='newmm', tgt_tok_type='newmm',
#               src_dict_path=src_dict_path, tgt_dict_path=tgt_dict_path,
#               model_path=model_path,
#               examples=texts,
#               n_examples=300,
#               beam_size=5)


run_inference(src_lang='th', tgt_lang='en',
              src_tok_type='newmm', tgt_tok_type='newmm',
              src_dict_path=src_dict_path, tgt_dict_path=tgt_dict_path,
              model_path=model_path,
              examples=texts_test_set,
              n_examples=1000,
              beam_size=5)

model_path ../data/opensubtitles_model/exp003-1/transformer_base/checkpoint10.pt
src_dict_path ../data/opensubtitles_bin/newmm-newmm/th-en/dict.th.txt
tgt_dict_path ../data/opensubtitles_bin/newmm-newmm/th-en/dict.en.txt
| loading model from ../data/opensubtitles_model/exp003-1/transformer_base/checkpoint10.pt
args {'no_progress_bar': True, 'log_interval': 500, 'log_format': 'simple', 'tensorboard_logdir': 'data/opensubtitles_model/exp003-1/transformer_base/tensorboard_log', 'tbmf_wrapper': False, 'seed': 1, 'cpu': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'criterion': 'label_smoothed_cross_entropy', 'tokenizer': 'space', 'bpe': None, 'optimizer': 'adam', 'lr_scheduler': 'inverse_sqrt', 'task': 'translation', 'num_workers': 12, 'skip_invalid_size_inputs_valid_test': False, 'max_tokens': 3000, 'max_sentences': None, 'requir


Target      : Because all of it sounds like me .
Target toks : Because all of it sounds like me .

Prediction 1: That ' s why it was there . </s>
Prediction 2: That ' s why it was there . That ' s why it was there . </s>
Prediction 3: That ' s why it was there . That ' s what it was like . </s>
Prediction 4: That ' s why it was there , that ' s what it was all about . </s>
Prediction 5: That ' s why it was there . That ' s what it was like to be there . </s>



12
Source : - หย่อน ลง ไป
tokens : ['-', 'หย่อน', 'ลง', 'ไป']

Target      : - Lower away !
Target toks : - Lower away !

Prediction 1: - Set </s>
Prediction 2: - Relax . </s>
Prediction 3: - Easy . </s>
Prediction 4: - Set . </s>
Prediction 5: - Lower . </s>



13
Source : ผม ไม่เคย เห็น คุณ ชอบ ดนตรี เลย
tokens : ['ผม', 'ไม่เคย', 'เห็น', 'คุณ', 'ชอบ', 'ดนตรี', 'เลย']

Target      : I ' ve never heard you express an interest in music .
Target toks : I ' ve never heard you express an interest in music .

Prediction 1: I never s


Target      : Your back , now , please !
Target toks : Your back , now , please !

Prediction 1: Turn ! </s>
Prediction 2: Turn around </s>
Prediction 3: Turn around ! </s>
Prediction 4: Turn around . </s>
Prediction 5: Turn it around </s>



30
Source : เฮ้ เธอ มองเห็น ฉัน ใช่ไหม
tokens : ['เฮ้', 'เธอ', 'มองเห็น', 'ฉัน', 'ใช่ไหม']

Target      : Hey ... you can see me , can ' t you ?
Target toks : Hey ... you can see me , can ' t you ?

Prediction 1: You ' re gonna get it . </s>
Prediction 2: You ' re gonna get me . </s>
Prediction 3: You can ' t get that . </s>
Prediction 4: You ' re gonna get it ! </s>
Prediction 5: You ' re gonna get me out of there . </s>



31
Source : เดียต กับ ลูกทีม จะ กอบกู้ ห่อ ของ
tokens : ['เดียต', 'กับ', 'ลูกทีม', 'จะ', 'กอบกู้', 'ห่อ', 'ของ']

Target      : Dietz and his team will retrieve a package .
Target toks : Dietz and his team will retrieve a package .

Prediction 1: They ' re gonna wrap it up . </s>
Prediction 2: He ' s going to wrap it up . </s


Target      : I fucked up royally . I ' m sorry .
Target toks : I fucked up royally . I ' m sorry .

Prediction 1: I ' m so fucked up . </s>
Prediction 2: I ' m so fucking fucked up . </s>
Prediction 3: I ' m so fucking fucked up ! </s>
Prediction 4: I ' m so fucked up , I ' m sorry . </s>
Prediction 5: I ' m so fucked up , I ' m so sorry . </s>



50
Source : ผม จะ ไป ประตู นั้น ได้ ยังไง ?
tokens : ['ผม', 'จะ', 'ไป', 'ประตู', 'นั้น', 'ได้', 'ยังไง', '?']

Target      : How do I get there ?
Target toks : How do I get there ?

Prediction 1: How could I ? </s>
Prediction 2: How can I ? </s>
Prediction 3: How could I be ? </s>
Prediction 4: How can I be ? </s>
Prediction 5: How can I be that ? </s>



51
Source : มี รอยบาก ของ กระสุน
tokens : ['มี', 'รอยบาก', 'ของ', 'กระสุน']

Target      : The bad back .
Target toks : The bad back .

Prediction 1: There ' s a charge . </s>
Prediction 2: There ' s a hand . </s>
Prediction 3: There ' s a print . </s>
Prediction 4: There ' s a cream . </s


Target      : Whoa . You okay ?
Target toks : Whoa . You okay ?

Prediction 1: Whoa . </s>
Prediction 2: Whoa ! </s>
Prediction 3: Whoa , you . </s>
Prediction 4: Whoa , whoa . </s>
Prediction 5: Whoa , whoa , whoa . </s>



65
Source : ขอ พูด ไว้ เคย ว่า ต้อง ทำ
tokens : ['ขอ', 'พูด', 'ไว้', 'เคย', 'ว่า', 'ต้อง', 'ทำ']

Target      : - I assure you I do .
Target toks : - I assure you I do .

Prediction 1: I ' m telling you , I ain ' t gonna pay for that . </s>
Prediction 2: I ' m telling you , I ain ' t gonna pay for that shit . </s>
Prediction 3: I ' m telling you , I ain ' t gonna pay for this shit . </s>
Prediction 4: I ' m telling you , I ain ' t gonna pay you for that shit . </s>
Prediction 5: I ' m telling you , I ' m telling you , I ' m gonna pay for that shit . </s>



66
Source : ปิด ตา ซะ มัน ไหม้ ได้
tokens : ['ปิด', 'ตา', 'ซะ', 'มัน', 'ไหม้', 'ได้']

Target      : Close your eyes . This stuff burns .
Target toks : Close your eyes . This stuff burns .

Prediction 1: Eyes .

...


Target      : Focus . You know this shit . Come on .
Target toks : Focus . You know this shit . Come on .

Prediction 1: Your intentions , your actions , these answers , </s>
Prediction 2: Your intentions , your knowledge , these answers , </s>
Prediction 3: Your intentions , your knowledge , these answers ... </s>
Prediction 4: Your intentions , your knowledge , your knowledge , </s>
Prediction 5: Your intentions , your knowledge , your knowledge , your actions . </s>



926
Source : อีก แก้ว
tokens : ['อีก', 'แก้ว']

Target      : Have another glass .
Target toks : Have another glass .

Prediction 1: It ' s a good one . </s>
Prediction 2: I ' m gonna have a drink . </s>
Prediction 3: I ' m gonna get a drink . </s>
Prediction 4: I ' m gonna get a drink , </s>
Prediction 5: It ' s a good one , man . </s>



927
Source : กองทัพ ฝูง ใหญ่ กำลัง เดินทัพ มา ทาง ชายฝั่ง
tokens : ['กองทัพ', 'ฝูง', 'ใหญ่', 'กำลัง', 'เดินทัพ', 'มา', 'ทาง', 'ชายฝั่ง']

Target      : Big mountainous bombs marchi


Target      : Well , I don ' t know if I ' d call it dumb .
Target toks : Well , I don ' t know if I ' d call it dumb .

Prediction 1: Uh , I don ' t think I ' m supposed to call it a <unk> . </s>
Prediction 2: Uh , I don ' t think I ' m supposed to call that a <unk> . </s>
Prediction 3: Uh , I don ' t know if I ' m supposed to call it a <unk> . </s>
Prediction 4: Uh , I don ' t think I ' m supposed to call it a " <unk> ." </s>
Prediction 5: Uh , I don ' t know if I ' m supposed to call it a " <unk> ." </s>



938
Source : ห้าม พก มีดโกน นะ
tokens : ['ห้าม', 'พก', 'มีดโกน', 'นะ']

Target      : No razor blades .
Target toks : No razor blades .

Prediction 1: You can ' t carry it . </s>
Prediction 2: You can ' t carry a pair . </s>
Prediction 3: You can ' t be carrying a muscle . </s>
Prediction 4: You can ' t even carry a pair . </s>
Prediction 5: You can ' t even carry a piece . </s>



939
Source : ทำไม
tokens : ['ทำไม']

Target      : Why ?
Target toks : Why ?

Prediction 1: Why ar


Target      : I know . I ' m sorry .
Target toks : I know . I ' m sorry .

Prediction 1: I know . </s>
Prediction 2: I ' m sorry . </s>
Prediction 3: I know , I ' m sorry </s>
Prediction 4: I know , I ' m sorry . </s>
Prediction 5: I know . I ' m sorry . </s>



953
Source : โอเค แต่ นั่น มัน เยอะ อยู่ นะ
tokens : ['โอเค', 'แต่', 'นั่น', 'มัน', 'เยอะ', 'อยู่', 'นะ']

Target      : Ok . That ' s a lot of chips .
Target toks : Ok . That ' s a lot of chips .

Prediction 1: Okay , but that ' s ... </s>
Prediction 2: Okay , but that ' s it . </s>
Prediction 3: Okay , but that ' s just ... </s>
Prediction 4: Okay , but that ' s just it . </s>
Prediction 5: Okay , but that ' s where it is . </s>



954
Source : ก็ เดา ๆ ซะ ที สิ ไอ้งั่ง ไม่มี แฟน !
tokens : ['ก็', 'เดา', 'ๆ', 'ซะ', 'ที', 'สิ', 'ไอ้งั่ง', 'ไม่มี', 'แฟน', '!']

Target      : Well , guess something , you girlfriendless geek !
Target toks : Well , guess something , you girlfriendless geek !

Prediction 1: Well , guess what . </s


Target      : I can ' t leave him .
Target toks : I can ' t leave him .

Prediction 1: I left behind . </s>
Prediction 2: I was left behind . </s>
Prediction 3: He was left behind . </s>
Prediction 4: I was left behind behind . </s>
Prediction 5: I was left behind behind behind . </s>



970
Source : ก็ดี
tokens : ['ก็ดี']

Target      : Great . Mmm .
Target toks : Great . Mmm .

Prediction 1: It ' s okay . </s>
Prediction 2: I ' m sorry . </s>
Prediction 3: It ' s just ... </s>
Prediction 4: I don ' t know . </s>
Prediction 5: I don ' t think so . </s>



971
Source : ช่าย
tokens : ['ช่าย']

Target      : Yeah .
Target toks : Yeah .

Prediction 1: I ' m sorry . </s>
Prediction 2: It ' s okay . </s>
Prediction 3: It ' s just ... </s>
Prediction 4: I don ' t know . </s>
Prediction 5: I don ' t think so . </s>



972
Source : บางที เธอ น่าจะ เขียน เกี่ยว กัน ฉัน น่ะ
tokens : ['บางที', 'เธอ', 'น่าจะ', 'เขียน', 'เกี่ยว', 'กัน', 'ฉัน', 'น่ะ']

Target      : Maybe you should just write it a


Target      : Tell me . I want to know .
Target toks : Tell me . I want to know .

Prediction 1: Tell me about it . </s>
Prediction 2: Tell me about that . </s>
Prediction 3: Tell me about it . I need to know . </s>
Prediction 4: Tell me about it . I ' m curious . </s>
Prediction 5: Tell me about it . I think I need to know . </s>



990
Source : ข้า คิด ว่า ข้า สามารถ ทำลาย ยาน พวก นั้น เพื่อที่จะ ให้ อาจารย์ โอ บี วัน สามารถ ผ่าน ไป ได้
tokens : ['ข้า', 'คิด', 'ว่า', 'ข้า', 'สามารถ', 'ทำลาย', 'ยาน', 'พวก', 'นั้น', 'เพื่อที่จะ', 'ให้', 'อาจารย์', 'โอ', 'บี', 'วัน', 'สามารถ', 'ผ่าน', 'ไป', 'ได้']

Target      : I thought I could knock out those ships so {\ when } Master Obi-Wan {\ arrived , he } could get through .
Target toks : I thought I could knock out those ships so {\ when } Master Obi-Wan {\ arrived , he } could get through .

Prediction 1: I thought I was worthy I could destroy the ships as I could destroy the ship as well as I thought I was to destroy the enemy ' s path . </s

In [45]:
# # exp003-4.1 th-en sentencepiece-sentencepiece

# model_path = '../data/opensubtitles_model/exp003-4/transformer_base/checkpoint4.pt'
# src_dict_path = '../data/opensubtitles_bin_sentencepiece_bpe/th-en/dict.th.txt'
# tgt_dict_path = '../data/opensubtitles_bin_sentencepiece_bpe/th-en/dict.en.txt'

# print('model_path', model_path)
# print('src_dict_path', src_dict_path)
# print('tgt_dict_path', tgt_dict_path)

# run_inference(src_lang='th', tgt_lang='en',
#               src_tok_type='sentencepiece', tgt_tok_type='sentencepiece',
#               src_dict_path=src_dict_path, tgt_dict_path=tgt_dict_path,
#               model_path=model_path,
#               examples=texts_sample,
#               n_examples=100,
#               beam_size=5)